<a href="https://colab.research.google.com/github/putrinahampun/NeuralNetworkArchitecture-ImageNet/blob/main/EfficientNet-Anemia-TrialandError/EfficientNet_Anemia01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Pre Processing

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
file_path = '/content/drive/My Drive/dataset-anemia'

In [51]:
from PIL import Image
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import tensorflow_hub as hub

# Fungsi untuk memuat gambar, mengonversi ke RGB, meresize, dan menormalisasikan
def load_images_and_preprocess(folder_path, target_size=(224, 224)):
    images = []
    labels = []

    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        for filename in os.listdir(label_path):
            img_path = os.path.join(label_path, filename)
            img = Image.open(img_path)

            # Convert gambar menjadi RGB
            img_rgb = convert_to_rgb(np.array(img))

            # Resize gambar
            img_resized = resize_image(img_rgb, target_size)

            # Normalisasi gambar
            img_normalized = normalize_image(img_resized)

            images.append(img_normalized)
            labels.append(label)

    return images, labels

# Fungsi untuk mengubah menjadi RGB
def convert_to_rgb(image):
    # Mengambil hanya 3 kanal pertama (R, G, B)
    rgb_image = image[:, :, :3]
    return rgb_image

# Fungsi untuk meresize gambar
def resize_image(image, size):
    img_resized = Image.fromarray(image).resize(size)
    return np.array(img_resized)

# Fungsi untuk normalisasi gambar
def normalize_image(image):
    normalized_image = image / 255.0
    return normalized_image

# Contoh penggunaan
folder_path = '/content/drive/My Drive/dataset-anemia'
target_size = (224, 224)
images, labels = load_images_and_preprocess(folder_path, target_size)

# Konversi label menjadi angka
label_dict = {'Anemic': 0, 'Non-anemic': 1}
numeric_labels = [label_dict[label] for label in labels]

# Konversi label menjadi one-hot encoding
categorical_labels = to_categorical(numeric_labels)

# Split data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(images, categorical_labels, test_size=0.2, random_state=42)

# Convert list gambar menjadi numpy array
X_train_array = np.array(X_train)
X_test_array = np.array(X_test)

# Load model EfficientNet dari TensorFlow Hub
module_url = "https://tfhub.dev/google/efficientnet/b0/classification/1"
model = tf.keras.Sequential([
    hub.KerasLayer(module_url, input_shape=(224, 224, 3))
])

# Tambahkan layer Dense untuk klasifikasi binary
model.add(tf.keras.layers.Dense(2, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Pelatihan model
model.fit(X_train_array, y_train, epochs=10, batch_size=32, validation_data=(X_test_array, y_test))


Epoch 1/10
18/18 [==============================] - 52s 3s/step - loss: 0.7729 - accuracy: 0.5739 - val_loss: 0.6482 - val_accuracy: 0.6268
Epoch 2/10
18/18 [==============================] - 47s 3s/step - loss: 0.6443 - accuracy: 0.6549 - val_loss: 0.6229 - val_accuracy: 0.6690
Epoch 3/10
18/18 [==============================] - 47s 3s/step - loss: 0.5961 - accuracy: 0.7113 - val_loss: 0.6772 - val_accuracy: 0.6268
Epoch 4/10
18/18 [==============================] - 48s 3s/step - loss: 0.5657 - accuracy: 0.7201 - val_loss: 0.5949 - val_accuracy: 0.7394
Epoch 5/10
18/18 [==============================] - 44s 2s/step - loss: 0.5366 - accuracy: 0.7729 - val_loss: 0.5892 - val_accuracy: 0.7042
Epoch 6/10
18/18 [==============================] - 46s 3s/step - loss: 0.5955 - accuracy: 0.6919 - val_loss: 0.7275 - val_accuracy: 0.6056
Epoch 7/10
18/18 [==============================] - 49s 3s/step - loss: 0.5175 - accuracy: 0.7676 - val_loss: 0.5887 - val_accuracy: 0.7465
Epoch 8/10
18/18 [==

In [77]:
from google.colab import files

uploaded = files.upload()

# Pilih satu atau beberapa gambar untuk diuji
for filename in uploaded.keys():
    img_path = filename
    uploaded_img = Image.open(img_path)


Saving Image_063.png to Image_063.png


In [78]:
# Convert gambar menjadi RGB
uploaded_imgs = convert_to_rgb(np.array(uploaded_img))

# Resize gambar
uploaded_img_resized = resize_image(uploaded_imgs, target_size)

# Normalisasi gambar
uploaded_img_normalized = normalize_image(uploaded_img_resized)

# Menambah dimensi agar sesuai dengan input model
uploaded_img_input = np.expand_dims(uploaded_img_normalized, axis=0)


In [79]:
# Melakukan prediksi
predictions = model.predict(uploaded_img_input)

# Mendapatkan kelas prediksi (0 atau 1)
predicted_class = np.argmax(predictions)

# Mendapatkan label kelas
class_labels = {0: 'anemia', 1: 'non-anemia'}
predicted_label = class_labels[predicted_class]

print("Hasil Prediksi:", predicted_label)


1/1 [==============================] - 0s 79ms/step
Hasil Prediksi: non-anemia
